In [10]:
import methods
import numpy as np
MIN_COUNT = 8


In [9]:
a=np.array([1,2,1]).T
b=np.array([1,3,1]).T
c=np.array([1,3,2]).T

In [2]:
class Sphere:
    """
    Class for a sphere object containing a triangle. Member variables are:
    v1, v2, v3 (3 element arrays) - the three vertices of the triangle
    center (3 element array) - the center of the sphere
    radius (float) - the radius of the sphere
    """
    def __init__(self, v1, v2, v3):
        """
        Creates Sphere object based on vertices of triangle that should be contained by Sphere
        :param v1: 3 element vector, a vertex of the triangle
        :param v2: 3 element vector, a vertex of the triangle
        :param v3: 3 element vector, a vertex of the triangle
        """
        self.v1 = v1
        self.v2 = v2
        self.v3 = v3
        center, self.radius = methods.CreateSphere(self.v1, self.v2, self.v3)
        self.center = center.ravel()


In [11]:
class BoundingBoxTreeNode:
    """
    Class for a node of the octree. Member variables are:
    spheres (list of Sphere objects) - list of spheres contained by this node
    nS (integer) - number of spheres contained by this node
    center (3 element array) - the average center of all spheres contained in this object
    max_radius (float) - the radius of the largest sphere contained by this node
    LB (3 element array) - the lower bound for each dimension to form the cube
    UB (3 element array) - the upper bound for each dimension to form the cube
    has_subtrees (boolean) - whether or not this node has children
    subtrees (8 element array of BoundingBoxTreeNode objects) - array containing all the children of this node
    """
    def __init__(self, bounding_spheres):
        """
        Creates a BoundingBoxTreeNode object based on a list of spheres
        :param bounding_spheres: list of Sphere objects to be contained in this node
        """
        self.spheres = bounding_spheres
        self.nS = len(self.spheres)
        self.center, self.max_radius, self.LB, self.UB = self.get_properties()
        self.has_subtrees = self.nS > MIN_COUNT
        self.subtrees = [None] * 8
        if self.has_subtrees:
            self.construct_subtrees()
            
    def construct_subtrees(self):
        """
        Constructs all the subtrees of this node
        """
        sphere_bins = self.split_sort()
        for i in range(8):
            self.subtrees[i] = BoundingBoxTreeNode(sphere_bins[i])
    def split_sort(self):
        """
        Partitions the list of spheres based on the location of each sphere in relation to the overall center
        :return: sphere_bins, a list containing 8 lists of spheres, forming a partition so that each list contains the
        spheres in one sub-region of the cube
        """
        sphere_bins = list()
        for i in range(8):
            sphere_bins.append(list())
        for sphere in self.spheres:
            bin_num = 0
            center = sphere.center
            if center[0] < self.center[0]:
                bin_num += 1
            if center[1] < self.center[1]:
                bin_num += 2
            if center[2] < self.center[2]:
                bin_num += 4
            sphere_bins[bin_num].append(sphere)
        return sphere_bins
    def find_closest_point(self, v, bound, closest):
        """

        :param v: 3 element array, the point we're searching
        :param bound: float scalar, the distance to the closest point so far
        :param closest: 3 element array, the closest point so far
        :return: updated bound and closest point
        """
        if self.nS == 0:
            return bound, closest
        dist = bound + self.max_radius
        print(v)
        print(bound)
        if (v[0] > (dist + self.UB[0]) or v[0] < self.LB[0] - dist or
                v[1] > dist + self.UB[1] or v[1] < self.LB[1] - dist or
                v[2] > dist + self.UB[2] or v[2] < self.LB[2] - dist):
            return bound, closest
        if self.has_subtrees:
            for subtree in self.subtrees:
                bound, closest = subtree.find_closest_point(v, bound, closest)
        else:
            for sphere in self.spheres:
                cp = methods.ClosestPointTriangle(sphere.v1, sphere.v2, sphere.v3, v)
                dist = np.linalg.norm(cp-v)
                if dist < bound:
                    bound = dist
                    closest = cp
        return bound, closest
    